In [1]:
%AddDeps org.bdgenomics.adam adam-core-spark2_2.11 0.24.0
%AddDeps comp.bio.aging adam-playground_2.11 0.0.12 --repository https://dl.bintray.com/comp-bio-aging/main/

Marking org.bdgenomics.adam:adam-core-spark2_2.11:0.24.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir7387851658533337832/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir7387851658533337832/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.jar
-> New file at /tmp/toree-tmp-dir7387851658533337832/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.pom


Waiting for a Spark session to start...

Marking comp.bio.aging:adam-playground_2.11:0.0.12 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir7387851658533337832/toree_add_deps/
-> https://dl.bintray.com/comp-bio-aging/main/
-> https://repo1.maven.org/maven2
Marking org.apache.hadoop:hadoop-azure:2.7.6 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir7387851658533337832/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir7387851658533337832/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.jar
-> New file at /tmp/toree-tmp-dir7387851658533337832/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.pom


Waiting for a Spark session to start...

In [2]:
import  org.apache.spark._
import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._

In [23]:
%AddDeps org.apache.hadoop hadoop-azure 2.7.6

Marking org.apache.hadoop:hadoop-azure:2.7.6 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir7387851658533337832/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir7387851658533337832/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.jar
-> New file at /tmp/toree-tmp-dir7387851658533337832/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.pom


lastException: Throwable = null


In [3]:
def sparkHadoopConf(sc: SparkContext, acountName: String, accountKey: String) = {
  sc.hadoopConfiguration.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
  sc.hadoopConfiguration.set("fs.azure.account.key." + acountName + ".blob.core.windows.net", accountKey)
  sc
}

sparkHadoopConf: (sc: org.apache.spark.SparkContext, acountName: String, accountKey: String)org.apache.spark.SparkContext


In [4]:
def azurize(container: String, accountName: String, blobFile: String): String = "wasbs://"+container+"@"+accountName+".blob.core.windows.net/"+blobFile 

def writeText2Azure[T]( rdd: RDD[T], container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  rdd.saveAsTextFile(url)
  url
}

def writeTsv2Azure( df: DataFrame, container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  df.write.option("sep","\t").option("header","true").csv(url)
  url
}

azurize: (container: String, accountName: String, blobFile: String)String
writeText2Azure: [T](rdd: org.apache.spark.rdd.RDD[T], container: String, accountName: String, blobFile: String)String
writeTsv2Azure: (df: org.apache.spark.sql.DataFrame, container: String, accountName: String, blobFile: String)String


In [5]:
val account = "pipelines1"
val key = "JHVfH9TNAcLhbIjVmIzT387Z2eAOFz1T0xvzJBb7z3jbWtMXspZD+E87OBDIvOyvnd8OWMdyPfKYSboGMKfIxQ=="
val connString = s"DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=${key};EndpointSuffix=core.windows.net"
def az(path: String): String = azurize("storage", account, path)

key = ==
connString = sDefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=${key}==;EndpointSuffix=core.windows.net
account = pipelines1


az: (path: String)String


JHVfH9TNAcLhbIjVmIzT387Z2eAOFz1T0xvzJBb7z3jbWtMXspZD+E87OBDIvOyvnd8OWMdyPfKYSboGMKfIxQ==

In [14]:
val sparkContext = sc
sparkHadoopConf(sparkContext, account, key)
  
val spark = SparkSession
  .builder()
  .appName("mapping_models")
  .getOrCreate()

sparkContext = org.apache.spark.SparkContext@77760c55
spark = org.apache.spark.sql.SparkSession@30a30d6d


Spark app-20180525142721-0011: Some(http://fc9b3fce6198:4040)

In [9]:
import org.apache.spark.sql.functions._
import spark.implicits._

val toDouble = udf[Double, String]( _.toDouble)

toDouble = UserDefinedFunction(<function1>,DoubleType,Some(List(StringType)))


UserDefinedFunction(<function1>,DoubleType,Some(List(StringType)))

In [10]:
val gtexPath = "/GTEx"
val genesPath = az(gtexPath + "/gtex_genes.tsv")

//val txtPath = az(gtexPath + "/GTEx_Analysis_2016-01-15_v7_RSEMv1.2.22_transcript_tpm.txt")
//val gctPath = az(gtexPath + "/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct")


gtexPath = /GTEx
genesPath = wasbs://storage@pipelines1.blob.core.windows.net//GTEx/gtex_genes.tsv


wasbs://storage@pipelines1.blob.core.windows.net//GTEx/gtex_genes.tsv

In [11]:
def readTSV(path: String, header: Boolean = false, sep: String = "\t"): DataFrame = spark.read
    .option("sep", sep)
    .option("comment", "#")
    .option("inferSchema", true)
    .option("header", header)
    .option("ignoreLeadingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("maxColumns", 150000)
    .csv(path)

readTSV: (path: String, header: Boolean, sep: String)org.apache.spark.sql.DataFrame


In [28]:
%AddDeps com.microsoft.azure azure-storage 7.0.0


Marking com.microsoft.azure:azure-storage:7.0.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir7387851658533337832/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir7387851658533337832/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/7.0.0/azure-storage-7.0.0.jar
-> New file at /tmp/toree-tmp-dir7387851658533337832/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/7.0.0/azure-storage-7.0.0.pom


In [29]:
val genes = readTSV(genesPath, true)

genes = [Name: string, ENSG00000223972.4: string ... 56201 more fields]


error: error while loading KVIterator, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-unsafe_2.11-2.3.0.jar(org/apache/spark/unsafe/KVIterator.class)' has location not matching its contents: contains class KVIterator
error: error while loading UnsafeAlignedOffset, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-unsafe_2.11-2.3.0.jar(org/apache/spark/unsafe/UnsafeAlignedOffset.class)' has location not matching its contents: contains class UnsafeAlignedOffset
error: error while loading Platform, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-unsafe_2.11-2.3.0.jar(org/apache/spark/unsafe/Platform.class)' has location not matching its contents: contains class Platform
error: error while loading UTF8String, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-unsafe_2.11-2.3.0.jar(org/apache/spark/unsafe/types/UTF8String.class)' has location not matching its contents: contains class UTF8String
error: error while loading CalendarInterval, class fi

[Name: string, ENSG00000223972.4: string ... 56201 more fields]

In [30]:
genes.count

error: error while loading Cross, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/Cross.class)' has location not matching its contents: contains class Cross
error: error while loading NaturalJoin, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/NaturalJoin.class)' has location not matching its contents: contains class NaturalJoin
error: error while loading LeftExistence, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/LeftExistence.class)' has location not matching its contents: contains class LeftExistence
error: error while loading RightOuter, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/RightOuter.class)' has location not matching its contents: contains class RightOuter
error: error while loading 

11689